### PDF Text Extraction

In [2]:
import pymupdf

In [7]:
doc = pymupdf.open("a guide to modern cookery.pdf")
out = open("output.txt", "wb")
i = 0
for page in doc:
    i += 1
    if i > 200:
        text = page.get_text().encode("utf8")
        out.write(text)
    if i == 20:
        print(text)
    if i == 230:
        break 
out.close()

b'204  GUIDE  TO  MODERN  COOKERY \n560\xe2\x80\x94 CONSOMMjg  AUX  DIABLOTINS \nCut  a  French  soup  "  flute  "  into  twelve  slices  one-quarter \ninch  thick.  Reduce  about  one-quarter  pint  of  Bechamel  to  a \nthick  consistence;  add  thereto,  away  from  the  fire,  two  heaped \ntablespoonfuls  of  grated  Gruy^re  cheese,  and  season  with  a  little \ncayenne. \nGarnish  the  slices  of  soup  "  flute  "  with  this  preparation, \narranged  in  the  form  of  a  dome,  upon  a  tray,  and  set  it  to \nglaze  a  few  minutes  before  serving. \nPour  one  quart  of  chicken  consomm^  into  the  soup-tureen, \nand  add  the  diablotins. \n561\xe2\x80\x94 CONSOMME  DIPLOMATE \nRoll  into  small  sausage-form  three  oz.  of  chicken  forcemeat, \nfinished  with  crayfish  butter.  Poach  the  sausages,  cut  them \ninto  thin  roundels,  and  put  them  into  the  soup-tureen  with  one \ndessertspoonful  of  very  black  truffle,  cut  in  julienne-fashion. \nPour

Fantastic, the data seems to be well extracted and follow a consistent structure. Recipe titles are in all caps followed by instructions. 